In [1]:
from __future__ import print_function
from matplotlib import pyplot as plt
# get_ipython().magic(u'matplotlib auto')
import tensorflow as tf
# import tensorflow.compat.v1 as tf
# tf.disable_v2_behavior()
# import torch
from tensorflow import keras
import statistics

In [2]:
# pip install -q -U tensorflow-addons

In [3]:
# import tensorflow_addons as tfa
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Activation
from tensorflow import reshape
from keras.utils import np_utils
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D,Conv2D, MaxPooling1D,AveragePooling1D
from tensorflow.keras.layers import SeparableConv1D, Bidirectional
from tensorflow.keras.layers import LocallyConnected2D,LocallyConnected1D
from tensorflow.keras.layers import ZeroPadding2D,ZeroPadding1D, MaxPooling2D, Bidirectional
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.layers import BatchNormalization,Add,concatenate
from tensorflow.keras.callbacks import CSVLogger,LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint

In [4]:
from scipy.io import loadmat
import numpy as np
import pandas as pd
import seaborn as sns
from numpy import mean
from numpy import std
# import coremltools
# from torch import nn, optim
# import torch.nn.functional as F
import scipy.io
from scipy import stats
#from IPython.display import display, HTML
from tensorflow.keras import optimizers
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import GRU
from tensorflow.keras.layers import TimeDistributed
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn import preprocessing

In [5]:
N_FEATURES = 8

In [6]:
column_names = ['C'+str(j) for j in range(1, N_FEATURES+1)]
lst = ['Class_label']
column_names = column_names+lst
print(column_names)

['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'Class_label']


In [7]:
def read_data_Train(file_path_Train):
    df_Train = pd.read_csv(file_path_Train,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Train['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Train['Class_label'] = df_Train['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Train.dropna(axis=0, how='any', inplace=True)
    return df_Train
def convert_to_float(x):
    try:
        return np.float(x)
    except:
        return np.nan

In [8]:
def show_basic_dataframe_info(dataframe):
    # Shape and how many rows and columns
    print('Number of columns in the dataframe: %i' % (dataframe.shape[1]))
    print('Number of rows in the dataframe: %i\n' % (dataframe.shape[0]))

In [9]:
def read_data_Test(file_path_Test):
    df_Test = pd.read_csv(file_path_Test,header=None,names=column_names)
    # Last column has a ";" character which must be removed ...
    df_Test['Class_label'].replace(regex=True,inplace=True,to_replace=r';',value=r'')
    # ... and then this column must be transformed to float explicitly
    df_Test['Class_label'] = df_Test['Class_label'].apply(convert_to_float)
    # This is very important otherwise the model will not fit and loss
    # will show up as NAN
    df_Test.dropna(axis=0, how='any', inplace=True)
    return df_Test

In [10]:
 pd.options.display.float_format = "{:,.5f}".format

In [11]:
# 15 function to segment data into trial lengths (trial length =513 samples in this dataset)
def create_segments_and_labels(df, time_steps,step,n_features, label_name):
    segments = []
    labels = []
    for i in range(0, len(df) - time_steps, step):
      for j in range(1, n_features+1):
        L = ('C'+str(j)) 
        segments.append(df[str(L)].values[i: i + time_steps])
      label = stats.mode(df[label_name][i: i + time_steps])[0][0]
      labels.append(label)
    reshaped_segments = np.asarray(segments, dtype= np.float32).reshape(-1, time_steps, n_features)
    labels = np.asarray(labels)
    return reshaped_segments, labels

In [12]:
# import os
path='/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/Total_process_TT/'

In [13]:
#  for i in range(num_res_net_blocks_64):
#     x = res_net_block_64(x, 64, 3)

In [14]:
# file_path_train=path+'Train_data'+'.'+'csv'
file_path_train=path+'UCI_6c_36s_olsson_train'+'.'+'csv'
print(file_path_train)

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/Total_process_TT/UCI_6c_36s_olsson_train.csv


In [15]:
file_path_test=path+'UCI_6c_36s_olsson_test'+'.'+'csv'
# file_path_test=path+'Test_data'+'.'+'csv'
print(file_path_test)

/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/olsson/Total_process_TT/UCI_6c_36s_olsson_test.csv


In [16]:
df_Train=read_data_Train(file_path_train)
show_basic_dataframe_info(df_Train)
df_Train.head(5)

Number of columns in the dataframe: 9
Number of rows in the dataframe: 972000



,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.00001,0.00000,-0.00001,0.00000,0.00000,-0.00001,-0.00001,0.00001,1.00000
1,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000
2,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000
3,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000
4,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000


In [17]:
df_Test=read_data_Test(file_path_test)
show_basic_dataframe_info(df_Test)
df_Test.head(5)

Number of columns in the dataframe: 9
Number of rows in the dataframe: 324000



,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
1,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
2,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
3,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
4,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000


In [18]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.00001,0.00000,-0.00001,0.00000,0.00000,-0.00001,-0.00001,0.00001,1.00000
1,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000
2,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000
3,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000
4,-0.00001,-0.00002,0.00000,-0.00001,-0.00001,-0.00001,-0.00003,-0.00002,1.00000


In [19]:
scaler = preprocessing.StandardScaler().fit(df_Train.iloc[:,0:N_FEATURES])

In [20]:
df_Train.iloc[:,0:N_FEATURES]=scaler.transform(df_Train.iloc[:,0:N_FEATURES])

In [21]:
pd.options.display.float_format = "{:,.5f}".format
df_Train.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.00918,0.07347,-0.00537,0.03266,0.05739,0.00585,-0.00049,0.10120,1.00000
1,-0.00918,-0.07394,0.06717,-0.00515,0.02545,0.00585,-0.11263,-0.04816,1.00000
2,-0.00918,-0.07394,0.06717,-0.00515,0.02545,0.00585,-0.11263,-0.04816,1.00000
3,-0.00918,-0.07394,0.06717,-0.00515,0.02545,0.00585,-0.11263,-0.04816,1.00000
4,-0.00918,-0.07394,0.06717,-0.00515,0.02545,0.00585,-0.11263,-0.04816,1.00000


In [22]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
1,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
2,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
3,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000
4,-0.00003,0.00002,-0.00003,-0.00001,0.00000,0.00000,0.00001,-0.00001,1.00000


In [23]:
df_Test.iloc[:,0:N_FEATURES]=scaler.transform(df_Test.iloc[:,0:N_FEATURES])

In [24]:
pd.options.display.float_format = "{:,.5f}".format
df_Test.head(5)

,C1,C2,C3,C4,C5,C6,C7,C8,Class_label
0,-0.11640,0.22089,-0.15047,-0.00515,0.05739,0.04570,0.11164,0.00163,1.00000
1,-0.11640,0.22089,-0.15047,-0.00515,0.05739,0.04570,0.11164,0.00163,1.00000
2,-0.11640,0.22089,-0.15047,-0.00515,0.05739,0.04570,0.11164,0.00163,1.00000
3,-0.11640,0.22089,-0.15047,-0.00515,0.05739,0.04570,0.11164,0.00163,1.00000
4,-0.11640,0.22089,-0.15047,-0.00515,0.05739,0.04570,0.11164,0.00163,1.00000


In [25]:
LABEL = 'ActivityEncoded'
# Transform the labels from String to Integer via LabelEncoder
le = preprocessing.LabelEncoder()
# Add a new column to the existing DataFrame with the encoded values
df_Train[LABEL] = le.fit_transform(df_Train['Class_label'].values.ravel())
# df_Valid[LABEL] = le.fit_transform(df_Valid['Class_label'].values.ravel())
df_Test[LABEL] = le.fit_transform(df_Test['Class_label'].values.ravel())

In [26]:
# The number of steps within one time segment
TIME_PERIODS = 200
# The steps to take from one segment to the next; if this value is equal to
# TIME_PERIODS, then there is no overlap between the segments
STEP_DISTANCE = 200

In [27]:
x_train, y_train = create_segments_and_labels(df_Train,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_train shape: ', x_train.shape)
# print(x_train)
print(x_train.shape[0], 'training samples')
print('y_train shape: ', y_train.shape)
# Set input & output dimensions
num_time_periods, num_sensors = x_train.shape[1], x_train.shape[2]
print('num_time_periods',num_time_periods)
print('num_sensors',num_sensors)
num_classes = le.classes_.size
print('class_list',list(le.classes_))
# input_shape = (num_time_periods,num_sensors)
# print(input_shape)
input_shape = (num_time_periods,num_sensors)
#x_train = x_train.reshape(x_train.shape[0], input_shape)
print('x_train shape:', x_train[0].shape)
print('input_shape:', input_shape)
x_train = x_train.astype('float32')
# x_train = [torch.tensor(arr, dtype=torch.float32) for arr in x_train]
# y_train = y_train.astype('float32')
# print(y_train)
y_train_hot = np_utils.to_categorical(y_train, num_classes)
print(y_train_hot)
# y_train_hot= [torch.tensor(arr, dtype=torch.uint8) for arr in y_train_hot]
print('New y_train shape: ', y_train_hot.shape)

x_train shape:  (4859, 200, 8)
4859 training samples
y_train shape:  (4859,)
num_time_periods 200
num_sensors 8
class_list [1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
x_train shape: (200, 8)
input_shape: (200, 8)
[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 1.]]
New y_train shape:  (4859, 6)


In [28]:
x_test, y_test = create_segments_and_labels(df_Test,TIME_PERIODS,STEP_DISTANCE,N_FEATURES,LABEL)
print('x_test shape: ', x_test.shape)
# print(x_train)
print(x_test.shape[0], 'testing samples')
print('y_test shape: ', y_test.shape)
# Set input_shape / reshape for Keras
#x_test = x_test.reshape(x_test.shape[0], input_shape)
x_test = x_test.astype('float32')
y_test = y_test.astype('float32')
y_test_hot = np_utils.to_categorical(y_test, num_classes)

x_test shape:  (1619, 200, 8)
1619 testing samples
y_test shape:  (1619,)


In [29]:
# n_steps, n_length = 20, 25
# n_steps, n_length= 10, 50
# n_steps, n_length= 16, 32
n_length =  200
n_depth=8
n_channel=8
x_train = x_train.reshape(x_train.shape[0], n_length,n_depth)
print('x_train shape: ', x_train.shape)
# x_valid = x_valid.reshape(x_valid.shape[0], n_steps, n_length, n_depth)
# print('x_valid shape: ', x_valid.shape)
x_test = x_test.reshape(x_test.shape[0],  n_length,n_depth)
print('x_test shape: ', x_test.shape)
n_outputs = y_train_hot.shape[1]
print('n_outputs',n_outputs)

x_train shape:  (4859, 200, 8)
x_test shape:  (1619, 200, 8)
n_outputs 6


In [30]:
def conv_block(input_data,n_channel):
    inputs=input_data
    print(inputs.shape)
    l_y=[]
    for i in range(0,n_channel):
        ip=inputs[:,:,i]
        print('ip_shape',ip.shape)
#         ip1=tf.convert_to_tensor(ip, dtype=tf.float32)
        ip=tf.reshape(ip,(tf.shape(ip)[0],200,1))
        print('ip_shape',ip.shape)
        x = Conv1D(64, kernel_size=3,kernel_initializer="he_normal",kernel_regularizer=l1(1e-04), \
               padding='same',input_shape=ip.shape)(ip)
        print('conv1_o',x.shape)
        x = BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x = Activation('relu')(x)
        x= Conv1D(filters=64, kernel_size=3,padding="same", kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1)(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        x= LocallyConnected1D(64, kernel_size=1, kernel_regularizer=l2(5e-04),kernel_initializer="he_normal",strides=1,padding='valid')(x)
        x= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(x)
        x= Activation('relu')(x)
        y= Dropout(0.5)(x)
        l_y.append(y)
    y_stack=tf.stack(l_y, axis=2)   
    return y_stack

In [31]:
def generate_model():
    inputs = Input(shape=(n_length,n_depth),name="main")
    p=conv_block(inputs,8)
    print(p.shape)
    print(type(p))
#     print(y.shape)
    y=Flatten()(p)
    print(y.shape)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(512, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    y= Dropout(0.5)(y)
    y= Dense(128, kernel_initializer="he_normal")(y)
    y= BatchNormalization(epsilon=1e-06, momentum=0.9, weights=None)(y)
    y= Activation('relu')(y)
    outputs=Dense(n_outputs, activation='softmax')(y)
    print(outputs.shape)
#     wei_mscnn_model = Model(inputs,sub_inputs], outputs)
    print('inputs_shape',inputs.shape)
    final_model = Model(inputs,outputs,name="wei_mscnn_model")
    return final_model        

In [32]:
if __name__ == "__main__":
    model = generate_model()

(None, 200, 8)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
ip_shape (None, 200)
ip_shape (None, 200, 1)
conv1_o (None, 200, 64)
(None, 200, 8, 64)
<class 'tensorflow.python.keras.engine.keras_tensor.KerasTensor'>
(None, 102400)
(None, 6)
inputs_shape (None, 200, 8)


In [38]:
verbose, epochs, batch_size = 0, 50, 32

In [39]:
def step_decay(epoch):
   initial_lrate = 1e-3
   drop = 0.1
   epochs_drop = 50.0
   lrate = initial_lrate * tf.math.pow(drop,  
           tf.math.floor((1+epoch)/epochs_drop))
   return lrate

In [40]:
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)
test_acc=[]

In [41]:
adam=optimizers.Adam(lr=1e-3, beta_1=0.9, beta_2=0.999, amsgrad=False)
# sgd=tfa.optimizers.SGDW(weight_decay=0.0001,learning_rate=0.1, momentum=0.9, nesterov=False, name='SGDW')
sgd=tf.optimizers.SGD(learning_rate=1e-3, momentum=0.9, nesterov=False, name='SGD')
checkpoint_filepath = '/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/wei/CNN25X20/checkpoint.hdf5'
# model.load_weights(checkpoint_filepath) 
checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath,verbose=1, monitor='val_accuracy',save_weights_only=True,save_best_only=True)
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=100, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
print(model.summary())
#     tf.keras.utils.plot_model(model, to_file='/media/naveen/nav/mat_codes/nina_DB1_codes/nina_prep_python/olsson/CNN20X10/Model1.png',show_shapes=True,show_layer_names=True,dpi=96)

Model: "wei_mscnn_model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main (InputLayer)               [(None, 200, 8)]     0                                            
__________________________________________________________________________________________________
tf.__operators__.getitem (Slici (None, 200)          0           main[0][0]                       
__________________________________________________________________________________________________
tf.__operators__.getitem_2 (Sli (None, 200)          0           main[0][0]                       
__________________________________________________________________________________________________
tf.__operators__.getitem_4 (Sli (None, 200)          0           main[0][0]                       
____________________________________________________________________________________

In [42]:
csv_logger = CSVLogger('/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/wei/CNN25X20/CNN_nina_20X10.csv', append=True, separator=';')
history = model.fit(x_train, y_train_hot, epochs=epochs, batch_size=batch_size, callbacks=[csv_logger,checkpoint_callback,lrate,early],validation_data=(x_test, y_test_hot), verbose=1)

Epoch 1/50
152/152 [==============================] - 95s 234ms/step - loss: 1.2347 - accuracy: 0.7962 - val_loss: 0.9251 - val_accuracy: 0.8468

Epoch 00001: val_accuracy improved from -inf to 0.84682, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/wei/CNN25X20/checkpoint.hdf5
Epoch 2/50
152/152 [==============================] - 15s 100ms/step - loss: 0.9673 - accuracy: 0.8167 - val_loss: 0.8393 - val_accuracy: 0.8487

Epoch 00002: val_accuracy improved from 0.84682 to 0.84867, saving model to /media/naveen/nav/mat_codes/EMG_data_for_gestures-master/wei/CNN25X20/checkpoint.hdf5
Epoch 3/50
152/152 [==============================] - 15s 99ms/step - loss: 0.8102 - accuracy: 0.8498 - val_loss: 0.8069 - val_accuracy: 0.8388

Epoch 00003: val_accuracy did not improve from 0.84867
Epoch 4/50
152/152 [==============================] - 16s 103ms/step - loss: 0.7758 - accuracy: 0.8468 - val_loss: 0.7304 - val_accuracy: 0.8598

Epoch 00004: val_accuracy improved from 0

152/152 [==============================] - 15s 102ms/step - loss: 0.4743 - accuracy: 0.9256 - val_loss: 0.6706 - val_accuracy: 0.8746

Epoch 00037: val_accuracy did not improve from 0.87956
Epoch 38/50
152/152 [==============================] - 15s 101ms/step - loss: 0.4505 - accuracy: 0.9379 - val_loss: 0.7356 - val_accuracy: 0.8419

Epoch 00038: val_accuracy did not improve from 0.87956
Epoch 39/50
152/152 [==============================] - 16s 102ms/step - loss: 0.4647 - accuracy: 0.9242 - val_loss: 0.7088 - val_accuracy: 0.8684

Epoch 00039: val_accuracy did not improve from 0.87956
Epoch 40/50
152/152 [==============================] - 15s 102ms/step - loss: 0.4846 - accuracy: 0.9261 - val_loss: 0.6872 - val_accuracy: 0.8653

Epoch 00040: val_accuracy did not improve from 0.87956
Epoch 41/50
152/152 [==============================] - 15s 101ms/step - loss: 0.4236 - accuracy: 0.9439 - val_loss: 0.6858 - val_accuracy: 0.8629

Epoch 00041: val_accuracy did not improve from 0.87956
Ep

In [43]:
best_index = history.history['val_accuracy'].index(max(history.history['val_accuracy']))
print('epoch_number',best_index+1)
print('train accuracy and validation accuracy', history.history['accuracy'][best_index], history.history['val_accuracy'][best_index])

epoch_number 44
train accuracy and validation accuracy 0.9362008571624756 0.8838789463043213


In [44]:
model.load_weights(checkpoint_filepath) 
_, testaccuracy = model.evaluate(x_test, y_test_hot, batch_size=batch_size, verbose=1)
print('test_accuracy',testaccuracy)
test_acc.append(testaccuracy)
print(test_acc)

51/51 [==============================] - 1s 28ms/step - loss: 0.6552 - accuracy: 0.8839
test_accuracy 0.8838789463043213
[0.8838789463043213]


In [45]:
statistics.mean(test_acc)

0.8838789463043213

In [46]:
fopen = open("/media/naveen/nav/mat_codes/EMG_data_for_gestures-master/wei/CNN25X20/wei_test.csv","a")
y_pred_test = model.predict(x_test)
# Take the class with the highest probability from the test predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test = np.argmax(y_test_hot, axis=1)
for i in range(0, len(max_y_pred_test)):
    fopen.writelines(str(max_y_test[i])+","+str(max_y_pred_test[i])+"\n")
show_confusion_matrix(max_y_test, max_y_pred_test)
print(classification_report(max_y_test, max_y_pred_test))
fopen.close()

NameError: name 'show_confusion_matrix' is not defined